In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pathlib
import umap

In [2]:
nELISA_data_path = pathlib.Path("./nELISA_Data_UCA1_2023.04.11.csv")
MetaData_path = pathlib.Path("./Metadata_UCA1_2023.04.11.csv")


nELISA_data = pd.read_csv(nELISA_data_path)
MetaData = pd.read_csv(MetaData_path)

In [5]:
MetaData.head()
nELISA_data.head(10)
nELISA_data = nELISA_data.filter(like="pgML", axis=1)

nELISA_data = nELISA_data.replace("Low_BC", 0)
nELISA_data = nELISA_data.replace("<LLOD", 0)
nELISA_data = nELISA_data.replace(">ULOD", 0)
nELISA_data

,Activin A [pgML],AITRL (GITR Ligand) [pgML],Amphiregulin [pgML],Amyloid beta [pgML],APRIL [pgML],BAFF [pgML],BCMA (TNFRSF17) [pgML],BDNF [pgML],BMP2 [pgML],BMP3 [pgML],...,TWEAK [pgML],uPA [pgML],VCAM-1 [pgML],VEGF Receptor 2 (Flk-1) [pgML],VEGF-A (165) [pgML],VEGF-C [pgML],VEGF-D [pgML],VEGFR-1 [pgML],WISP-1 (CCN4) [pgML],XCL1 (Lymphotactin) [pgML]
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,0,0,0,0,0,0,0,0,0,0,...,0,1883.94005637406,0,0,129.865293358785,0,134.079279475407,0,0,0
764,0,0,0,0,0,0,0,0,0,0,...,0,4114.98691346606,0,0,169.911439118221,0,0,0,0,0
765,0,0,0,0,0,0,0,0,0,0,...,0,2472.65466070024,2065.65905995158,0,151.830135822137,0,0,0,103.689421684213,0
766,0,0,0,0,0,0,0,0,0,0,...,0,0,2065.87696661985,0,124.224474418766,0,0,0,0,0
